<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Томография

## Examples – Biophysics
<section class="post-meta">
By Jonas Tjemsland, Håkon Ånes, Andreas Krogen og Jon Andreas Støvneng.
</section>
Last edited: January 20th 2019

___
This notebook is based on an assignment given in the course *TMA4320 Introduction to Scientific Computation* at NTNU in April 2016 [[1]](#rsc). The assignment was prepared by Pål Erik Goa, Jon Andreas Støvneng, Peder Galteland and Grunde Wesenberg. The code is based on the answers by Gjert Magne Knutsen, Daniel Halvorsen and Jonas Tjemsland.
___

## Вступление

Медицинская визуализация занимает центральное место в современной диагностике. [Рис. 1](#img1) ниже показано поперечное сечение головы, основанное на так называемой *компьютерной томографии* (КТ). В этом блокноте мы получаем представление о том, как компьютерная томография использует рентгеновские лучи для восстановления довольно четких изображений. Мы будем использовать изображения в оттенках серого размером $N\times N$, которые представлены матрицей $N\times N$. Значение каждого элемента матрицы будет представлять "серость" пикселя ($1$ для белого и $0$ для черного). В конце мы будем использовать инструменты, разработанные на уроке [Фильтрация изображений с использованием преобразования Фурье](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/image_filtering_using_fourier_transform.ipynb) чтобы получить более четкое изображение.
<a id="img1"></a>
<body>
<img src="https://upload.wikimedia.org/wikipedia/en/0/04/Brain_CT_scan.jpg"  width=250px, alt="CT scan of the brain"/>  

**Рисунок 1:** *Аксиальная компьютерная томография головного мозга [[2]](#rsc).*

Чтобы полностью понять и оценить результаты, мы начнем с теории. Нетерпеливому читателю рекомендуется перейти к разделу *Простой пример*.

In [ ]:
import matplotlib.pyplot as plt
import progressbar
import numpy as np
import scipy.fftpack as fft
from skimage import io
import math
import warnings

In [ ]:
warnings.filterwarnings("ignore")
%matplotlib inline

# Set some figure parameters
newparams = {'axes.labelsize': 9, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 1, 'figure.figsize': (8, 3),
             'ytick.labelsize': 7, 'xtick.labelsize': 7,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 9, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 9, 'figure.dpi': 200,
             'mathtext.fontset': 'stix', 'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

## Теория

### Рентген

Рентгеновские лучи представляют собой электромагнитное излучение в диапазоне длин волн от $0.01$ до $10\:\mathrm{нм}$, соответствующее энергиям между $100\:\mathrm{эВ}$ и $100\:\mathrm{кэВ}$. Фотоны могут, например, генерироваться в вакуумных лампах. В вакуумной трубке электроны испускаются с нагретого катода и ускоряются к аноду электрическим полем, создаваемым ускоряющим высоким напряжением. Излучение может создаваться различными физическими механизмами. Например, ускоренный электрон может выбить электрон с одной из внутренних орбиталей атома анода. Электрону, находящемуся в состоянии с более высокой энергией, затем разрешается занять это пустое состояние и по пути туда испустить фотон. Этот фотон имеет характеристическую энергию, равную разности энергий между двумя электронными орбиталями, или состояниями, в атоме анода. Такой переход из второго низшего состояния в состояние с наименьшей энергией называется $K_\alpha$, и его энергия приблизительно описывается *законом Мозли*,

$$
E(K_\alpha)=10.2 \:\mathrm{eV}\cdot (Z-1)^2,
$$

где $Z$ - атомный номер материала анода. Вольфрам ($W; Z=74$) дает энергию $E(K_\alpha) = 54.4\:\mathrm{keV}$, немного ниже экспериментального значения $59.3\:\mathrm{keV}$.

В дополнение к этому характерному рентгеновскому излучению наблюдается непрерывный спектр с четко определенной минимальной длиной волны. Это происходит из-за *разрушающего излучения* или *тормозного излучения* и происходит, по существу, во всех ситуациях, когда заряженные частицы ускоряются в электрических полях, создаваемых другими частицами. В рентгеновской трубке ускоренные электроны замедляются, когда они сталкиваются с атомами в материале анода. Данный электрон может максимально передать всю свою кинетическую энергию $E_k$ одному фотону. Это означает, что возбужденный фотон имеет длину волны, большую или равную

$$
\lambda_{min}=\frac{hc}{E_k},
$$

где $E_k = \mathrm{eV}$ определяется напряжением $V$ между катодом и анодом. Здесь $h$ - постоянная Планка, $\mathrm{e}$ - элементарный заряд, а $c$ - скорость света.

В действительности КТ использует фотоны как от характерного излучения, так и от тормозного излучения. Для простоты мы предположим, что фотон имеет определенную энергию, скажем, 60 кэВ, что примерно совпадает с излучением $K_\alpha$ от вольфрама.

### Затухание

Пусть объект (например, человеческая голова) облучается рентгеновскими лучами (или любым другим типом электромагнитного излучения) с заданной интенсивностью $I$. Часть рентгеновских фотонов пройдет через объект без изменений. Остальные фотоны будут либо поглощены, либо рассеяны атомами. Вероятность поглощения или рассеяния будет сильно зависеть как от энергии фотона, так и от характеристик материала объекта. Для заданной энергии фотона относительное уменьшение интенсивности $\mathrm{d}I/I$ на единицу длины $\mathrm{d}t$ , имеет вид

$$
\frac{\mathrm{d}I/I}{\mathrm{d}t}=-\mu,
$$

где $\mu$ - *коэффициент линейного затухания*. $\mu$ является высоким для твердых и плотных тканей, низким для мягких тканей (обычно с высоким содержанием воды) и приблизительно нулевым для воздуха. Для фотонов с энергией $60\:\mathrm{keV}$ у нас будет $\mu\approx 600\: \mathrm{m}^{-1}$ для костей и $\mu\approx 20\:\mathrm{m}^{-1}$ для мягких тканей.

Интенсивность $I$, прохождения через детектор, получается из приведенного выше уравнения,

$$
I=I_0\exp\left\{-\int_{t_1}^{t_2}\mu(t)\;\mathrm{d}t\right\}.
$$

Этот интеграл называется *проекцией*. Его значение будет зависеть от того, где рентгеновский луч попадает на объект, обозначенный $s$, и как объект ориентирован относительно направления луча, обозначенного $\theta$.

$$
p(s,\theta)=\ln(I_0/I)=\int_{t_1}^{t_2}\mu(t)\mathrm{d}t.
$$

Обратите внимание, что $\mu$, $t_1$ и $t_2$ зависят от $s$ и $\theta$, как показано на [рис. 2](#fig2) ниже.

## Установка

Рентгеновские лучи посылаются тонкими (по существу одномерными) лучами в направлении объекта, подлежащего отображению. Используя несколько лучей, расположенных рядом, мы можем, по существу, изучать горизонтальное поперечное сечение объекта. Мы собираемся проанализировать, как информация об объекте может быть извлечена из таких экспериментов. Другими словами, как мы можем реконструировать поперечное сечение?

<a id="fig2"></a>
<img src="images/tomography_setup.png" width=500px, alt="Source and detector schematic diagram.">

**Рисунок 2:** *Источник и детектор повернуты на угол $\theta$ относительно объекта. Координата $s$ указывает, где рентгеновский луч (пунктирные линии) попадает на объект. Координата $t$ определяет направление луча. Для $\theta=0$, $t$ совпадает с осью $y$. Луч входит в объект при $t=t_1$ и покидает объект при $t=t_2$. Функция $f(x,y)$ представляет распределение различных видов тканей внутри объекта. В общем, мы можем думать о $f(x,y)$ как о цвете в оттенках серого в позиции $(x,y)$.*

Из [Рис. 2](#рис. 2) мы получаем соотношение

$$
s=x\cos\theta + y\sin \theta,
$$

таким образом, линия проекции для конкретных значений $s$ и $\theta$ может быть записана как 

$$
y(x;s,\theta)=-x\cot\theta +\frac{s}{\sin\theta}.
$$

## Выполнение измерения

Измеряя проекции для $s$-значений, которые охватывают весь объект, и для $\theta$-значений, которые охватывают все возможные ориентации объекта ($\theta \geq 180^\circ$), мы получаем матрицу $\{p(s,\theta)\}$ – так называемую *синограмму*. Синограмма содержит в принципе всю информацию, необходимую для реконструкции распределения тканей объекта $f(x,y)$. Но чтобы извлечь эту информацию, мы должны использовать форму томографической реконструкции.

## Получение обратной проекции

Из синограммы мы можем построить *прямую обратную проекцию* $g(x,y)$. Заданная проекция $p(s,\theta)$ дает нам значение интеграла $f(x,y)$ на прямой, которая попадает на объект в положении $s$ для заданной ориентации $\theta$. Лучшее, что мы можем сделать (без дополнительной информации о $f(x, y)$), - это предположить, что значение интеграла равномерно распределено между $t_1$ и $t_2$, учитывая, что

$$
g(x,y;s,\theta)=\frac{p(s,\theta)}{t_2-t_1}.
$$

Это, конечно, можно сделать для всех значений $\theta$ и $s$. Если мы позволим $x$ и $y$ быть фиксированными и добавим все значения $g(x,y;s,\theta)$, мы получим $g(x, y)$. Если это сделать для всех $(x,y)$, мы получим прямое обратное проецируемое изображение.

Как вы, возможно, догадались, обратное проецируемое изображение будет размытой версией исходного изображения. Кроме того, сейчас, возможно, было бы неплохо сделать паузу и взглянуть на изображения, представленные ниже, прежде чем продолжить чтение, чтобы визуализировать то, что было сказано до сих пор.

## Простой пример

Мы начнем с простого примера на сетке $3\times 3$, описываемой матрицей $3\times 3$. Рассмотрим объект,состоящий из квадратной кости ( $f(x, y)=1$; белый), окруженной мягкой тканью ( $f(x,y)=0$; черный). Мы начинаем с генерации синограммы с использованием углов $\theta=\{0,\pi/2\}$. В разделе "Получение обратной проекции" выше, посмотрите, можете ли вы найти ответ (или, по крайней мере, визуализировать ответ) на эти вопросы:

* Каковы три проекции при $\theta = 0$? То есть $p(m,0)$ с $m=1$ для 1 столбца и т.д.
* Каковы три проекции при $\theta = \pi/2$? То есть $p(m,\pi/2)$ с $m=1$ для первой строки и т.д.
* Каково изображение, которое вы получаете при прямой обратной проекции из этих шести проекций?

Теперь мы выполняем вычисления и строим график результата.

In [ ]:
f = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
m, n = np.shape(f)

# Calculate the six projections for theta={0,pi/2}
pRow = np.sum(f, 1) # theta = pi/2
pCol = np.sum(f, 0) # theta = 0

# Calculate the direct back projection
g = np.zeros((m, n))
for row in range(0, m):
    g[row, :] = pRow[row]/m + pCol[:]/n

# Normalise (strictly not necessary)
g = np.sum(f)*g/np.sum(g)

# Visualise the result
plt.subplot(121)
plt.imshow(f, cmap='gray', interpolation='nearest')
plt.title('Model object');
plt.subplot(122)
plt.imshow(g, cmap='gray', interpolation='nearest')
plt.title('Back projection');

## Создание синограммы

Для простоты мы будем считать, что исходное изображение имеет $N\times N$ пикселей, где $N$ равно 2.

Теперь мы создаем функцию, которая вычисляет синограмму для произвольного $N$ и количества углов $\theta$. Пусть количество проекций равно $2N-1$ (количество диагоналей), так что каждый пиксель подсчитывается только один раз, когда $\theta = \pi/4$. Для произвольного $\theta$ мы выбираем, чтобы каждый пиксель использовался только в одной проекции.

Трудная часть состоит в том, чтобы выяснить, какой пиксель $(i, j)$ принадлежит данной проекции $(s,\theta)$. Это делается путем проверки того, какая проекция ближе всего к данному пикселю. Конечно, есть несколько способов сделать это.

In [ ]:
def sinogram(img, Ntheta):
    """Create a sinogram of an image.
    
    Parameters:
        img:    array-like, shape (N, N). Image.
        Ntheta: int. Number of projection angles in radians between 0 and pi/2
    Returns: 
        NumPy-array, shape (2*N-1, Ntheta). Sinogram.
    """
    M, N = np.shape(img)
    assert(M == N)
    Ndiag = 2*N - 1;               # Number of projections (diagonals)
    p = np.zeros((Ndiag, Ntheta)); # Allocate memory for the sinogram
    # Create a meshgrid for the pixels (x - N/2, y - N/2)
    A = (2*np.arange(1, N + 1) - N - 1)/2
    [x, y] = np.meshgrid(A, A)
    # Calculate the projection for Ntheta number of angles (theta) between 0 and pi/2,
    # which is stored in the sinogram, p
    bar = progressbar.ProgressBar()
    for k in bar(range(0, Ntheta)):
        theta = k*np.pi/Ntheta # Current angle
        # Find the projection number to each pixel for the given angle
        m = np.round(N + np.sqrt(2)*(x*np.cos(theta) + y*np.sin(theta)))
        # Iterate through each projection and put the pixels in the right place in
        # the sinogram
        for i in range(0, N):
            for j in range(0, N):
                p[int(m[i][j]) - 1][k] = p[int(m[i][j] - 1), k] + img[i][j]; # Add to sinogram
    return p

## Создание прямой обратной проекции

Теперь мы создаем функцию,которая вычисляет прямую обратную проекцию $g(x, y)$. Вклад данной проекции $p(s_m,\theta_n)$ в пиксель $(i,j)$ равен $p(s_m,\theta_n)/M$, где $M$ - количество пикселей, принадлежащих этой проекции.

In [ ]:
def back_projection(p):
    """ Create a direct back projection of a sinogram created in a similar
    fashion as in the sinogram() function above. The original image is assumed
    to be an N x N matrix.
    
    Parameters:
        p: array-like, shape (2*N-1, Ntheta). Sinogram.
    Returns: 
        NumPy-array, shape (N, N). Sinogram.
    """
    Ndiag, Ntheta = np.shape(p) # Number of projections and projection angles
    N = int((Ndiag + 1)/2)      # The size of the original image (assumed N x N)
    g = np.zeros((N, N))        # Allocate memory for the back projection
    # Create a meshgrid for the pixels (x - N/2, y - N/2)
    A = (2*np.arange(1, N + 1) - N - 1)/2
    [x, y] = np.meshgrid(A, A)
    # Compute the back projection
    bar = progressbar.ProgressBar()
    for k in bar(range(0, Ntheta)):
        theta = k*np.pi/Ntheta  # Current angle
        # The projection number to each pixel for the given angle
        m = np.round(N + np.sqrt(2)*(x*np.cos(theta) + y*np.sin(theta)));
        m = np.array(m, 'int')
        # Compute a vector holding the number of pixels belonging to each
        # of the Ndiag projections
        M = np.zeros(Ndiag);
        for i in range(0, int(N)):
            for j in range(0, int(N)):
                M[m[i, j] - 1] += 1
        # Iterate through each pixel and add the corresponding projection
        # value divided by the number of pixels in this projection
        for i in range(0, int(N)):
            for j in range(0, int(N)):
                g[i][j] += p[m[i][j] - 1, k]/M[m[i][j] - 1]
    # Divide by the number of projections, such that the total pixel value
    # of the original image and the direct back projection are equal
    g = g/Ntheta
    return g

## Пример – известное исходное изображение

### Создание синограммы и прямой обратной проекции

Теперь у нас есть необходимые инструменты для создания синограммы из (в оттенках серого) изображения $(N, N)$, где $N=2$. Давайте приведем пример со знаменитым фантомом Шеппа-Логана [[3]](#rsc). Изображение $128\times 128$ можно найти [здесь](https://www.numfys.net/media/notebooks/images/phantom_128.png) и изображение $256 \times 256$, можно найти [здесь](https://www.numfys.net/media/notebooks/images/phantom_256.png).

In [ ]:
f = io.imread('images/phantom_256.png', as_grey=True)

In [ ]:
p = sinogram(f, 180)

In [ ]:
g = back_projection(p)

In [ ]:
# Plot original image
plt.subplot(131)
plt.imshow(f, cmap='gray', interpolation='nearest')
plt.title('Original image, $f(x,y)$')

# Plot sinogram
plt.subplot(132)
m, n = np.shape(p)
plt.imshow(p, cmap='gray', aspect=n/m, interpolation='nearest')
plt.title(r'Sinogram, $p(s,\theta)$')

# Plot direct back projection
plt.subplot(133)
plt.imshow(g, cmap='gray', interpolation='nearest')
plt.title('Direct back projection, $g(x,y)$');

### Оценка погрешности

Теперь, когда известно исходное изображение, мы можем оценить ошибку в прямой обратной проекции. Мы будем использовать *среднеквадратичное отклонение* (среднеквадратичное значение) в качестве оценки ошибки

$$
RMS=\frac{1}{n}\sqrt{\sum_{\{i,j\}}^n\left[f(i,j)-g(i,j)\right]^2}.
$$

In [ ]:
def RMS(f, g):
    """ Calculate the root mean square deviation of two N x N numpy arrays f and g. """
    return np.sqrt(np.sum((f - g)**2))/len(f)

In [ ]:
print('Error estimate: RMS = %.5f%%' % (RMS(f, g)*100))

Пока и ошибка, и визуальный результат довольно плохие, и трудно распознать какие-либо детали.

### Фильтрация результата

Теперь мы можем отфильтровать прямую обратную проекцию и попытаться получить более четкое изображение. Это может быть достигнуто с помощью инструментов, разработанных в вышеупомянутом блокноте [фильтрация изображений](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/image_filtering_using_fourier_transform.ipynb).

In [ ]:
def FFT(image): return np.fft.fftshift(np.fft.fft2(image))

def IFFT(spectrum): return np.fft.ifft2(np.fft.fftshift(spectrum))

def filter_spectrum(spectrum, filter_type=None, val=50):
    n, m = np.shape(spectrum)
    y, x = np.meshgrid(np.arange(1, m + 1), np.arange(1, n + 1))
    R2 = ((x - n/2)**2 + (y - m/2)**2)
    if (filter_type == 'lowpass'):
        return spectrum*(R2 <= val**2)
    elif (filter_type == 'highpass'):
        return spectrum*(R2 >= val**2)
    elif (filter_type == 'gaussian_highpass'):
        return spectrum*(1 - np.exp(-val*R2))
    elif (filter_type == 'gaussian_lowpass'):
        return spectrum*np.exp(-val*R2)
    elif (filter_type != None):
        raise ValueError('%s is not a valid filter!' % filter_type)
    return spectrum

def visualise(image, spectrum, title='', title_img='', title_spec='', cmap='gray'):
    plt.subplot(121)
    plt.imshow(image, cmap=cmap)
    plt.title(title_img)
    plt.subplot(122)
    plt.imshow(np.log(abs(spectrum)), cmap=cmap)
    plt.title(title_spec)
    plt.suptitle(title)
    plt.show()

Нам нужно использовать фильтр высоких частот на изображении, чтобы получить более четкий результат; например, идеальный фильтр высоких частот или фильтр высоких частот Гаусса. Последнее дает более плавный результат, поэтому мы используем его. Поиграйте с различными фильтрами (и добавьте новые) и постарайтесь получить лучший результат!

In [ ]:
# Fourier transform of the back projection
spectrum = FFT(g)

# Visualise the result
visualise(g, spectrum, title_img='Input image', title_spec='Spectrum')

In [ ]:
filtered_spectrum = filter_spectrum(spectrum, 'gaussian_highpass', 0.01)
filtered_g = np.real(IFFT(filtered_spectrum))

# As a trick, we set all pixels below a specific value to zero to get
# a sharper result
val = 0
filtered_g = filtered_g*(filtered_g > val) + val*(filtered_g < val) - val

# Visualise the result
visualise(filtered_g, filtered_spectrum, title_img='Filtered image',
          title_spec='Filtered spectrum')

# "Normalise" the filtered image
filtered_g = filtered_g*np.sum(f)/np.sum(filtered_g)
# Print the error estimate
print(RMS(f, filtered_g)*100)
print('Error estimate: RMS = %.5f%%' % (RMS(f, filtered_g)*100))

Результат намного лучше, чем прямая обратная проекция, как визуально, так и в отношении среднеквадратичного отклонения.

## Пример – неизвестное исходное изображение

При реальной компьютерной томографии исходное изображение неизвестно, но синограмма есть. Теперь мы используем то, что мы узнали до сих пор, чтобы попытаться получить исходное изображение из синограммы. Мы сделали три синограммы доступными на нашем веб-сайте, чтобы вам было с чем поиграть: [sinogram_1.png](https://www.numfys.net/media/notebooks/images/sinogram_1.png), [sinogram_2.png](https://www.numfys.net/media/notebooks/images/sinogram_2.png), [sinogram_3.png](https://www.numfys.net/media/notebooks/images/sinogram_3.png).

In [ ]:
p = io.imread('images/sinogram_1.png')

In [ ]:
g = np.transpose(back_projection(p))

In [ ]:
# Plot the sinogram
plt.subplot(121)
m, n = np.shape(p)
plt.imshow(p, cmap='gray', aspect=n/m, interpolation='nearest')
plt.title(r'Sinogram, $p(s,\theta)$')

# Plot the direct back projection
plt.subplot(122)
plt.imshow(g, cmap='gray', interpolation='nearest')
plt.title('Direct back projection, $g(x,y)$');

Вы видите, кто это? Давайте выполним ту же фильтрацию, что и ранее, и посмотрим, сможем ли мы получить более четкое изображение.

In [ ]:
# Fourier transform of the back projection
spectrum = FFT(g)

# Filter the spectrum
filtered_spectrum = filter_spectrum(spectrum, 'gaussian_highpass', 0.01)

# Inverse Fourier Transform
filtered_g = np.real(IFFT(filtered_spectrum))

# Set all pixels with a negative value to zero
filtered_g = filtered_g*(filtered_g > 0)

# Set all pixels with a value over a certain value to val
val = 4
filtered_g = filtered_g*(filtered_g <= val) + val*(filtered_g > val)

# Visualise the result
plt.imshow(filtered_g, cmap='gray', interpolation='nearest');

Теперь вы видите, кто это?
___

## Дальнейшая работа и улучшения

* Что такое *теорема о проекционном срезе*? Как ее можно использовать для аргументации того, как выглядят фильтры?
* Как можно обобщить этот блокнот для работы с произвольным изображением $N \times M$?
* Как мы можем выполнить те же вычисления на круглом изображении?
* Мы использовали простые и интуитивно понятные алгоритмы для вычисления синограммы и прямой обратной проекции. Попробуйте найти какие-нибудь более сложные алгоритмы. Найдите, например, *преобразование радона* или проверьте [TomoPy](https://github.com/tomopy/tomopy).

<a id="rsc"></a>
## Источники и дальнейшее чтение

<a>[1]</a> Project description in Norwegian, https://wiki.math.ntnu.no/_media/tma4320/2016v/tomografi.pdf, acquired: 2016-10-21.

<a>[2]</a> A CT scan of an anonymous brain. Provided by Aaron G. Filler, MD, PhD. Image source: https://en.wikipedia.org/wiki/File:Brain_CT_scan.jpg

<a>[3]</a> Shepp, Larry; B. F. Logan (1974). *The Fourier Reconstruction of a Head Section*. IEEE Transactions on Nuclear Science. NS-21 (3): 21–43. Figure downloaded from https://commons.wikimedia.org/wiki/File:SheppLogan_Phantom.svg [acquired 2 Nov, 2016].